In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random
from lib.html_viewer import view_questions,view_components,p,h3,inpt
from lib.local_database import db,Question,Collection,db,Series,Database
from lib.svg_generator import Grid,svg,line
import json

In [3]:
db = Database()

In [4]:
collection = db.get_collection("math","C")

In [16]:
collection

<Collection math_C 26 types>

### Fläche

In [5]:
tip = (
    h3('Die Größe von Flächen bestimmen wir, indem wir fragen wie viele Centimeterquadrate die Fläche enthält.')+
    Grid(3,1,width="3cm").get_html()+
    h3('diese Fläche enthält 3 Quadrate und ist deshalb 3 quadratcentimeter (cm²) groß.')+
    Grid(5,2,width="5cm").get_html()+
    h3('diese Fläche ist 10 cm² groß.')
    
)
view_components(tip)

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [6]:
def gen_rect(x,y):


    v = max(x,y)+2
    g = Grid(v,v)

    title = h3("wie groß ist das blaue quadrat in cm²?")
    title += inpt + ""

    g.add_figure(
        (1,1),(x+1,1),(x+1,y+1),(1,y+1),(1,1),color = "blue", fill= "lightblue"
    )
    title = g.get_html() + title

    return Question.standart(title,x*y,[],tip)

view_questions(gen_rect(2,2))

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [7]:
q = [
    gen_rect(3,1),
    gen_rect(2,2),
    gen_rect(4,2),
    gen_rect(5,2),
]

In [8]:
view_questions(*q)

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [9]:
collection.add_series(q,4,"fläche bestimmen")

In [10]:
def get (data,x,y):
        w = len(data[0])
        h = len(data)
        if x<0 or y<0 or x >= w or y >= h:
            return ' '
        return data[y][x]


def draw_points(data):
    w = len(data[0])
    h = len(data)
    all_points = [[(x,y) for x in range(w)]for y in range(h)]

    border = set()
    
    for x in range(w+1):
        for y in range(h+1):
            me = get(data,x,y)
            if get(data,x-1,y) != me:
                border.add(((x,y),(x,y+1)))
            if get(data,x,y-1) != me:
                border.add(((x,y),(x+1,y)))

    border_line = []
    start,end = border.pop()

    while len(border) != 0:
        border_line.append(start)
        nx,ny = end
        options = [((nx,ny),(nx+1,ny)),
            ((nx,ny),(nx,ny+1)),
            ((nx-1,ny),(nx,ny)),
            ((nx,ny-1),(nx,ny)),
        ]
        
        found = False
        for opt in options:
            if opt in border:
                start = end
                if end == opt[0]:
                    end = opt[1]
                else:
                    end = opt[0]
                border.remove(opt)
                found = True

        if not found:
            break
    border_line.append(start)
    border_line.append(end)

    return border_line


def draw(data):
    w = len(data[0])
    h = len(data)

    
    border_line = draw_points(data)
    g = Grid(w,h)

    g.add_figure(*border_line,color="red",fill="lightcoral")
    return g.get_html(),len(border_line)-1

In [11]:
def gen_random_figure(a = 1,w=8,h=8):
    data = [[ ' ' for _ in range(w)]for _ in range(h)]

    data[int(h/2)][int(w/2)] = 'x'

    x = random.randint(0,w-1)
    y = random.randint(0,h-1)
    c = a-1
    while c > 0:
        x = random.randint(0,w-1)
        y = random.randint(0,h-1)
        if get (data,x,y) == ' ':
            for opt in [(x+1,y),(x-1,y),(x,y+1),(x,y-1)]:
                if get(data,*opt)=='x':
                    data[y][x] = 'x'
                    c-=1
                    break


    if not check_holes(data,w,h):return gen_random_figure(a,w,h)


    return data,a

def check_holes(data,w,h):
    def is_border(x,y):return x <0 or x>=w or y<0 or y>=h
    for x in range(w):
        for y in range(h):
            if get(data,x,y) == ' ':
                free = False
                dirs = [(0,1),(1,0),(0,-1),(-1,0)]
                free = False
                for dir in dirs:
                    pos = [x,y]
                    if is_border(*pos):
                        free = True
                        break
                    while not get(data,*pos)== 'x':
                        pos = [pos[0]+dir[0],pos[1]+dir[1]]
                        if is_border(*pos):
                            free = True
                            break
                    if free:break
                if not free:
                    return False
    return True


# html = gen_random_figure()
# view_components(html)
data,a = gen_random_figure(15)
html,_ = draw(data)
view_components(html)

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [12]:
def gen(a):
    fig,a= gen_random_figure(a,8,8)
    fig,_ = draw(fig)
    title = fig + h3('wie groß ist die markierte Fläche?')
    return Question.standart(title,a,[],tip = tip)


In [13]:
a_s = [2,3,4,5,6,6,7,7,8,8,9,9,10,10,11,12,12,13,13,15,15,16,16,17,17,18,19,]
q = list(map(gen,a_s))

In [14]:
view_questions(*q)

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [15]:
collection.add_series(q,6,'fläche bestimmen 2')

## Umfang

In [16]:
tip = h3('du kannst den Umfang einer Fläche bestimmen indem du zählst wie lang die Grenze der Fläche ist.')

g = Grid(6,4,'6cm')
g.add_figure((1,1),(5,1),(5,3),(1,3),(1,1),color = 'blue')

tip += g.get_html()
tip += p('4 + 2 + 4 + 2 = 12')
tip += p('In dieser Figur beträgt der Umfang 12cm.')

view_components(tip)

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [17]:
def gen(x,y):

    g = Grid(x+2,y+2,width = f"{x+2}cm")

    g.add_figure(
        (1,1),(x+1,1),(x+1,y+1),(1,y+1),(1,1),color = "blue", fill= "lightblue"
    )
    title = g.get_html() + h3('wie groß ist der Umfang dieser Figur?')
    return Question.standart(title,[x*2+y*2],[],tip)

q=gen(4,2)
view_questions(q)


view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [18]:
q = [
    gen(3,1),
    gen(4,1),
    gen(4,2),
    gen(5,2),
    gen(5,3),
]


In [19]:
def gen(a):
    data,_ = gen_random_figure(a,6,3)
    html,u=draw(data)

    title = html + h3('wie groß ist der Umfang dieser Figur?')
    return Question.standart(title,u,[],tip,)

In [20]:
q = [*map(gen,[4,5,6,6,7,7])]

In [21]:
mp = map (gen,[4,5,6,7,7])

In [22]:
q += [gen(4),gen(5),gen(6),gen(6),gen(7),gen(7)]

In [23]:
view_questions(*q)

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [24]:
collection.add_series(q,6,'umfang bestimmen')

## achsensymmetrie

In [25]:
tip = h3('achsensymmetrische figuren haben eine Spiegelachse.')
tip += h3('Sie teilt die Figur in zwei Hälften die gespiegelt gleich sind.')

g = Grid(16,8)

g.add_figure((2,6),(2,3),(1,3),(1,1),(2,1),(2,2),(6,2),(6,1),(7,1),(7,3),(6,3),(6,6),
            (5,6),(5,4),(3,4),(3,6),(2,6) ,fill='blue')

g.add_figure((4,0),(4,8),color='red')

g.add_figure((9,1),(15,1),(15,2),(13,2),(13,4),(14,4),(14,5),(13,5),(13,6),(12,6),(12,5),(10,5),
             (10,7),(9,7),(9,1), fill = 'green')

g.add_figure((8,0),(16,8),color='red')

tip += g.get_html()

tip += p('beide Figuren sind Achsensymmetrisch. Die Spiegelachsen sind in rot eingezeichnet.')

view_components(tip)

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [26]:
def stupiddraw(g:Grid,data,color='none',fill='blue'):
    w = len(data[0])
    h = len(data)
    for x in range(w):
        for y in range(h):
            if data[y][x] == 'x':
                points = [(x,y),(x+1,y),(x+1,y+1),(x,y+1),(x,y)]

                g.add_figure(*points,color = color,fill = fill)

In [60]:

def get_random_symetric_figure(a,w,h,axis):
        d,_ = gen_random_figure(a,w,h)

        if axis == 0:
            for y in range(8):
                for x in range(4):
                    x_ = 8-x-1
                    d[y][x] = d[y][x_]
        elif axis == 1:
            for x in range(8):
                for y in range(4):
                    y_ = 8-y-1
                    d[y][x] = d[y_][x]
        
        elif axis == 2:
            for x in range(8):
                for y in range(x):
                    d[x][y] = d[y][x]
        else:
            for x in range(8):
                for y in range(x):
                    d[7-x][y] = d[7-y][x]
            # diag
            
        return d

def gen(sym = True,symetric = get_random_symetric_figure):

    w,h = 8,8
    g = Grid(w,h)

    colors = random.sample(['red','blue','green','yellow'],3)
    axis = random.randint(0,3)
    data = [symetric(n,w,h,axis) for n in [16,10,6]]

    def flip():

        x,y = random.randint(0,w-1) ,random.randint(0,h-1)
        if x==y or x==(1-w-y):
            flip()
            return 
        hit = False
        for i in range(len(data)).__reversed__():
            if data[i][y][x] == 'x':
                hit = True
                data[i][y][x] = ' '
                break
        if not hit :flip()

    if not sym:
        for n in range(random.choice([1,3])):
            flip()

    for i in range(3):
        stupiddraw(g,data[i],fill=colors[i])

    html = g.get_html()
    
    if sym:
        c,w = 'Ja','Nein'
    else:
        c,w = 'Nein','Ja'
    return Question.multiple_choice(html+h3('ist diese Figur Achsensymmetrisch?'),[c],[w],tip)


q = gen(True)
view_questions(q)

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [28]:
q =[gen(True) for _ in range(10)]
q += [gen(False) for _ in range(10)]
q = random.sample(q,len(q))

In [29]:
view_questions(*q)

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [30]:
collection.add_series(q,5,'achsensymmetrie')

## drehsymmetrie

In [31]:
def get_random_rotation_symmetric_figure(a,w,h,rot_num):

    d,_ = gen_random_figure(a,w,h)
    rot_num %= 2

    if rot_num == 0:
        for y in range(8):
            for x in range(4):

                d[y][x] = d[7-y][7-x]
    elif rot_num == 1:
        for x in range(4):
            for y in range(4):

                if d[7-y][7-x]=='x' or d[y][x]=='x':
                    target = 'x'
                else:target =' '
                d[7-y][7-x] = target
                d[y][x] = target
                d[x][7-y] = target
                d[7-x][y] = target

    
    elif rot_num == 2:
        for x in range(8):
            for y in range(x):
                d[x][y] = d[y][x]
    else:
        for x in range(8):
            for y in range(x):
                d[7-x][y] = d[7-y][x]
    return d


def gen(sym = True,symetric = get_random_symetric_figure):

    w,h = 8,8
    g = Grid(w,h)

    colors = random.sample(['red','blue','green','yellow'],3)
    axis = random.randint(0,3)
    data = [symetric(n,w,h,axis) for n in [16,10,6]]

    def flip():

        x,y = random.randint(0,w-1) ,random.randint(0,h-1)
        if x==y or x==(1-w-y):
            flip()
            return 
        hit = False
        for i in range(len(data)).__reversed__():
            if data[i][y][x] == 'x':
                hit = True
                data[i][y][x] = ' '
                break
        if not hit :flip()

    if not sym:
        for n in range(random.choice([1,3])):
            flip()

    for i in range(3):
        stupiddraw(g,data[i],fill=colors[i])

    html = g.get_html()
    
    if sym:
        c,w = 'Ja','Nein'
    else:
        c,w = 'Nein','Ja'
    return Question.multiple_choice(html+h3('ist diese Figur Rotationssymmetrisch?'),[c],[w],'')



In [32]:
q = gen(True,symetric=get_random_rotation_symmetric_figure)


In [33]:
# raise Warning("check visual output for problematic content please!")
a = 10
q = [gen(True,symetric=get_random_rotation_symmetric_figure) for _ in range(a)]
q += [gen(False,symetric=get_random_rotation_symmetric_figure) for _ in range(a)]


In [34]:
view_questions(*q)

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [35]:
def change(n):
    q[n] = gen(n<10,symetric=get_random_rotation_symmetric_figure)

In [41]:
change(12)
cont = []
for i,item in enumerate(q):
    cont .append(item.title+str(i))
view_components(*cont)

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [42]:
q = random.sample(q,len(q))

In [43]:
view_questions(*q)

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [44]:
collection.add_series(q,6,'rotations symmetrie')

## körper

In [45]:
class vec3:
    def __init__(self,x:float,y:float,z:float):
        self.x,self.y,self.z = x,y,z
    def __sub__(self,other):
        return vec3(self.x - other.x,self.y-other.y,self.z-other.z)
    def __add__(self,other):
        return vec3(self.x + other.x,self.y+other.y,self.z+other.z)
    def __mul__(self,s):
        return vec3(self.x*s,self.y*s,self.z*s)
    def __repr__(self) -> str:
        return f'<{self.x,self.y,self.z}>'
class Quader:
    def __init__(self,pos:vec3,size:vec3,color='blue'):

        pos.y = pos.y
        size.y = size.y
        self.pos = pos
        self.size = size
        self.end  = pos + size
        self.color = color
    
    def img(self):
        p = 0.3
        l = self.pos.x + self.pos.z * p
        r = l + self.size.x
        b = self.pos.y + self.pos.z * p
        t = b + self.size.y
        d = self.size.z *p

        return self.view(p,l,r,b,t,d)
    
    def view(self,p,l,r,b,t,d):
        b = 1000 - b
        t = 1000 - t

        ln = line((r,b),(l,b),
                 (l,t),(r,t),(r,b)
                 ,fill = self.color)
        
        ln += line((r,t),(l,t),(l+d,t-d),(r+d,t-d),(r,t),fill=self.color)
        ln += line((r,t),(r,b),(r+d,b-d),(r+d,t-d),(r,t),fill=self.color)
        return ln
    
    def front_img(self):
        l = self.pos.x
        r = l + self.size.x
        b = self.pos.y 
        t = b + self.size.y
        return self.view(0,l,r,b,t,0)

    def right_img(self):
        l = self.pos.z
        r = l + self.size.z
        b = self.pos.y 
        t = b + self.size.y
        return self.view(0,l,r,b,t,0)
    
    def left_img(self):
        r = 1000 - self.pos.z
        l = r - self.size.z
        b = self.pos.y 
        t = b + self.size.y
        return self.view(0,l,r,b,t,0)

    def behind_img(self):
        r = 1000-self.pos.x
        l = r-self.size.x
        b = self.pos.y 
        t = b + self.size.y
        return self.view(0,l,r,b,t,0)
    
    def top_img(self):
        r = self.pos.x
        l = r+self.size.x
        b = self.pos.z
        t = b + self.size.z
        return self.view(0,l,r,b,t,0)
    
    def bottom_img(self):
        r = self.pos.x
        l = r+self.size.x
        b = 1000-self.pos.z
        t = b - self.size.z
        return self.view(0,l,r,b,t,0)
class Group :
    def __init__(self,*qs:Quader):
        self.qs = qs

    def mkview(self,key,img):
        qs = sorted(self.qs,key=key)
        html = ""
        for q in qs:
            html += img(q)()
        return svg(html)
    
    def view(self,dir='normal'):
        if dir == 'vorne':return self.mkview(lambda q: -q.pos.z,lambda q:q.front_img)
        if dir == 'links':return self.mkview(lambda q: -q.pos.x,lambda q:q.left_img)
        if dir == 'unten':return self.mkview(lambda q: -q.pos.y,lambda q:q.bottom_img)
        if dir == 'hinten':return self.mkview(lambda q: q.pos.z+q.size.z,lambda q:q.behind_img)
        if dir == 'rechts':return self.mkview(lambda q: q.pos.x+q.size.x,lambda q:q.right_img)
        if dir == 'oben':return self.mkview(lambda q: q.pos.y+q.size.y,lambda q:q.top_img)
        return self.mkview(lambda q : 1, lambda q: q.img)

In [46]:
g1 = Group(
    Quader(vec3(100,100,500),vec3(200,300,300),color='yellow'),
    Quader(vec3(100,100,100),vec3(300,200,400),color='blue'),
    Quader(vec3(400,100,100),vec3(300,100,200),color='red'),
)
g2 =Group(
    Quader(vec3(100,100,100),vec3(500,100,300),color='green'),
    Quader(vec3(100,200,100),vec3(200,300,200),color='red'),
    Quader(vec3(300,200,100),vec3(200,200,400),color='yellow')
)

g3 = Group(
    Quader(vec3(100,100,100),vec3(100,400,300),color='green'),
    Quader(vec3(600,100,100),vec3(100,400,300),color='red'),
    Quader(vec3(100,500,100),vec3(600,100,300),color='yellow'),
    Quader(vec3(300,600,100),vec3(200,100,300),color='blue'),
)

g4 = Group(
    Quader(vec3(300,100,400),vec3(400,100,100),color='red'),
    Quader(vec3(200,100,300),vec3(400,100,100),color='green'),
    Quader(vec3(400,200,200),vec3(100,100,500),color='blue'),
    Quader(vec3(400,300,400),vec3(100,200,100),color='yellow'),
)

g = [g1,g2,g3,g4]

In [47]:
view_dirs = ['vorne','hinten','rechts','links', 'oben','unten']
def gen(g:Group,dir,correct):

    normal_view = g.view('normal')

    options = view_dirs.copy()
    options.remove(dir)
    if correct:
        options = random.sample(options,2)
        options += ["nein"]
        correct = dir
    else:
        options = random.sample(options,2)
        options += [dir]
        correct = 'nein'

        #swap to colors
        # exclude red and green swap for colorblinds
        first = random.choice(list(filter(lambda q:q.color != 'red' and q.color != 'green',g.qs)))
        other = random.choice(list(filter(lambda q:q!=first,g.qs)))
        temp = first.color
        first.color=other.color
        other.color = temp
    
    ansicht = g.view(dir)
    
    img = '<div style="display:flex;width:80vh;margin:auto">{}</div>'.format(normal_view+ansicht)
    title = img+h3('zeigt das zweite bild eine Ansicht vom körper und wenn ja von welcher Richtung?')

    return Question.multiple_choice(title,[correct],options,"")

In [48]:
view_components(*map(lambda i : i.view(),g))

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [49]:
q = gen(g1,'vorne',False)
view_questions(q)

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [50]:
q = []
for group in g:
    for dir in view_dirs:
        q.append(gen(group,dir,random.choice([True,False])))


In [53]:
view_components(*map(lambda i:i[1].title+i[1].solution+str(i[0]),enumerate(q)))

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [52]:
# q.pop(13)

<Q:  <h3><div style="display:flex;width:80vh; : ["nein"]>

In [55]:
q = random.sample(q,len(q))

In [56]:
view_questions(*q)

view file:///Users/peergynt/fun/svelte/monsun/factory/views/index.html in browser


In [57]:
collection.add_series(q,6,'perspektivische ansichten')

# end

In [58]:
db.save()

sname: Wertetafel rstep: 0.5
sname: Große Nachbarzahlen rstep: 0.5
sname: Zahlen vergleichen rstep: 0.09999999999999964
sname: zahlenstrahl rstep: 1.0
sname: zahlenreihe rstep: 0.7142857142857153
sname: Runden rstep: 0.4800000000000004
sname: überschlag rstep: 0.25
sname: schriftliche addition rstep: 0.29999999999999716
sname: schriftliche addition ergänzen rstep: 0.20000000000000284
sname: schriftliche subtraktion rstep: 0.29999999999999716
sname: schriftliche subtraktion ergänzen rstep: 0.20000000000000284
sname: große multiplikation rstep: 0.028571428571424917
sname: große division rstep: 0.057142857142864045
sname: schriftliche multiplikation rstep: 0.29999999999999716
sname: schriftliche multiplikation zweistellig rstep: 0.20000000000000284
sname: runden bei mulitiplikation rstep: 0.09999999999999432
sname: schrifltiche division rstep: 0.30000000000001137
sname: runden bei division rstep: 0.12000000000000455
sname: schriftliche division mit rest rstep: 0.19999999999998863
sname: t

In [59]:
collection

<Collection math_C 32 types>

In [100]:
db = Database()

In [101]:
db.save()

sname: Wertetafel rstep: 0.5
sname: Große Nachbarzahlen rstep: 0.5
sname: Zahlen vergleichen rstep: 0.09999999999999964
sname: zahlenstrahl rstep: 1.0
sname: zahlenreihe rstep: 0.7142857142857153
sname: Runden rstep: 0.4800000000000004
sname: überschlag rstep: 0.25
sname: schriftliche addition rstep: 0.29999999999999716
sname: schriftliche addition ergänzen rstep: 0.20000000000000284
sname: schriftliche subtraktion rstep: 0.29999999999999716
sname: schriftliche subtraktion ergänzen rstep: 0.20000000000000284
sname: große multiplikation rstep: 0.028571428571424917
sname: große division rstep: 0.057142857142864045
sname: schriftliche multiplikation rstep: 0.29999999999999716
sname: schriftliche multiplikation zweistellig rstep: 0.20000000000000284
sname: runden bei mulitiplikation rstep: 0.09999999999999432
sname: schrifltiche division rstep: 0.30000000000001137
sname: runden bei division rstep: 0.12000000000000455
sname: schriftliche division mit rest rstep: 0.19999999999998863
sname: t